# SQL Alchemy INTRO Notebook

Introdução ao SQLAlchemy.

Baseada em:
[documentação] (https://docs.sqlalchemy.org/en/14/orm/tutorial.html#version-check)

In [2]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base,sessionmaker
from sqlalchemy import Column, Integer, String



# - Criando uma engine.
engine = create_engine('sqlite:///sdata.db', echo=True)
Base = declarative_base()


# Declarative Base

Constroi uma classe base para definições de classes declarativas.

A nova classe base receberá uma metaclasse que produz objetos Table apropriados e faz as chamadas mapper () apropriadas com base nas informações fornecidas declarativamente na classe e em quaisquer subclasses da classe.

No caso Abaixo, a nossa classe User vai receber uma nova declarative base, relacionada a Base acima.

In [3]:
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
            self.name, self.fullname, self.nickname)


In [4]:
User.__table__ 

Table('users', MetaData(), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

In [5]:

Base.metadata.create_all(engine)


2021-12-01 00:50:34,488 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-12-01 00:50:34,494 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2021-12-01 00:50:34,497 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-12-01 00:50:34,501 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
Session = sessionmaker(bind=engine)
Session.configure(bind=engine)
session = Session()

In [7]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')
session.add(ed_user)

In [8]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', nickname='windy'),
    User(name='mary', fullname='Mary Contrary', nickname='mary'),
    User(name='fred', fullname='Fred Flintstone', nickname='freddy')])


In [9]:
ed_user.nickname = 'eddie'

In [10]:
session.dirty

IdentitySet([])

In [11]:
session.commit()

2021-12-01 00:50:34,862 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-12-01 00:50:34,867 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2021-12-01 00:50:34,869 INFO sqlalchemy.engine.Engine [generated in 0.00142s] ('ed', 'Ed Jones', 'eddie')
2021-12-01 00:50:34,872 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2021-12-01 00:50:34,873 INFO sqlalchemy.engine.Engine [cached since 0.006085s ago] ('wendy', 'Wendy Williams', 'windy')
2021-12-01 00:50:34,875 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2021-12-01 00:50:34,877 INFO sqlalchemy.engine.Engine [cached since 0.009962s ago] ('mary', 'Mary Contrary', 'mary')
2021-12-01 00:50:34,879 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2021-12-01 00:50:34,882 INFO sqlalchemy.engine.Engine [cached since 0.01542s ago] ('fred', 'Fred Flintstone', 'freddy')
20

## Updating 

Para modificar valores, selecionar uma instancia e modificar o valor

In [12]:
ed_user.name='Eduardo'

Não esquecer de fazer o commit!

In [13]:
session.commit()

2021-12-01 00:50:35,076 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-12-01 00:50:35,084 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2021-12-01 00:50:35,086 INFO sqlalchemy.engine.Engine [generated in 0.00213s] (5,)
2021-12-01 00:50:35,092 INFO sqlalchemy.engine.Engine UPDATE users SET name=? WHERE users.id = ?
2021-12-01 00:50:35,094 INFO sqlalchemy.engine.Engine [generated in 0.00173s] ('Eduardo', 5)
2021-12-01 00:50:35,098 INFO sqlalchemy.engine.Engine COMMIT


In [14]:
session.flush()

In [15]:
fake_user = User(name='fakeuser', fullname='Invalid', nickname='12345')

In [16]:
session.add(fake_user)

In [22]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

2021-12-01 00:50:58,083 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users
2021-12-01 00:50:58,098 INFO sqlalchemy.engine.Engine [generated in 0.01513s] ()


[<User(name='Eduardo', fullname='Ed Jones', nickname='eddie')>,
 <User(name='wendy', fullname='Wendy Williams', nickname='windy')>,
 <User(name='mary', fullname='Mary Contrary', nickname='mary')>,
 <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>,
 <User(name='Eduardo', fullname='Ed Jones', nickname='eddie')>,
 <User(name='wendy', fullname='Wendy Williams', nickname='windy')>,
 <User(name='mary', fullname='Mary Contrary', nickname='mary')>,
 <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>]

In [ ]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

In [18]:
session.rollback()

2021-12-01 00:50:37,922 INFO sqlalchemy.engine.Engine ROLLBACK


In [19]:
ed_user.name

2021-12-01 00:50:38,330 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-12-01 00:50:38,338 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2021-12-01 00:50:38,340 INFO sqlalchemy.engine.Engine [generated in 0.00232s] (5,)


'Eduardo'

In [20]:
query = session.query(User).filter(User.name.like('%ed')).order_by(User.id)
query.all()

2021-12-01 00:50:38,955 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
2021-12-01 00:50:38,957 INFO sqlalchemy.engine.Engine [generated in 0.00253s] ('%ed',)


[<User(name='fred', fullname='Fred Flintstone', nickname='freddy')>,
 <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>]